In [ ]:
%run ./library/distributions_and_points_visualizations.py
%run ../utils/function_parameters.py

### Application

#### Visualize Points

In [ ]:
mean = [0, 0]
cov = [[1, 0], [0, 100]]
pts = np.random.multivariate_normal(mean, cov, size=800)

visualize_points(pts).show()

#### Visualize Distributions

In [ ]:
# each row has the values for mutlivariate normal distributions
distributions = np.array([
    [[3, 0], [1, 0], [0, 1]],
    [[2, 0], [1, 0.5], [0.5, 1]],
    [[0, 2], [1, -0.2], [-0.2, 1]],
    [[2, 1], [1, 0.5], [0.5, 1]],
    [[0, 2], [1, -1.2], [-0.2, 1]],
    [[-2, -2], [1, 0.3], [0.3, 1]]
])

plot_confidence_ellipses(distributions)[0].show()

In [ ]:
points = np.array([[-5.87064722, -1.3043999 ],
        [-4.46720547, -0.82537099],
        [ 0.2079403 , -2.52028114],
        [-7.47029182, -0.32820168],
        [-2.19769777, -5.08291467],
        [-0.41545212,  2.20838336],
        [ 1.4353041 ,  6.22338689],
        [-0.02220151, -5.98261222],
        [ 1.28872624,  6.99948287],
        [ 5.16801891,  4.13991605]])

points_to_distributions = np.array([2, 4, 5, 9, 6, 1])
plot_distributions_with_colored_points(distributions, points, points_to_distributions, n_std=1.96)[0].show()

### Visualize Distributions and Points

In [ ]:
points = np.random.uniform(-10, 10, (40, 2))

distributions_a = np.array([
    [[3, 4], [1, 0], [0, 1]],
    [[2, 0], [1, 0], [0, 1]]
])

distributions_b = np.array([
    [[4, 4], [1, 0], [0, 1]],
    [[3, 0], [1, 0], [0, 1]]
])

distributions_c = np.array([
    [[4, 0], [1, 0], [0, 1]],
    [[5, 4], [1, 0], [0, 1]]
])

distributions = [distributions_a, distributions_b, distributions_c]

weights = [0.7, 0.2, 0.1]

ids = [
    [0, 1],
    [0, 1],
    [1, 0]
]

plt = plot_gmm_distributions(distributions, weights, ids)
plt.show()

### Visualize Distributions and Points with mapping

In [ ]:
points = np.array([[ 2.67536762,  1.61771433],
        [-1.51995242,  8.5575738 ],
        [ 5.9708441 ,  2.88736993],
        [-0.26015255, -1.21744485],
        [-4.24165224,  3.16340135],
        [10.58514974, -1.02516812],
        [11.83292385, -1.69103837],
        [ 2.55579144,  3.80067187],
        [ 1.15372272, -2.601238  ],
        [ 6.47700649,  8.65276974]])

point_id_mapping = np.array([ 1, -1, -1, -1, -1, -1, -1,  0, -1, -1])

distributions_a = np.array([
    [[3, 4], [1, 0], [0, 1]],
    [[2, 0], [1, 0], [0, 1]]
])

distributions_b = np.array([
    [[4, 4], [1, 0], [0, 1]],
    [[3, 0], [1, 0], [0, 1]]
])

distributions_c = np.array([
    [[4, 0], [1, 0], [0, 1]],
    [[5, 4], [1, 0], [0, 1]]
])

distributions = [distributions_a, distributions_b, distributions_c]

weights = [0.7, 0.2, 0.1]

ids = [
    [0, 1],
    [0, 1],
    [1, 0]
]

plt = plot_gmm_distributions_with_points(distributions, weights, ids, points, point_id_mapping)
plt.show()